In [1]:
#Import Required Pacakges
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import pickle

In [2]:
# load Corpus Dataframe
#Open Corpus of News Article Text
with open('../news_articles_processing/data_frames/bitcoin_news_df_processed_for_modeling.pickle', 'rb') as file:
     bitcoin_news_df = pickle.load(file)

In [3]:
bitcoin_news_df.head(3)

,text,date,sent_with_num,tokenized_text,tfid_vec_text,stemmed_text,lemmatized_text,pos_text,tokenized_sent_with_num,pos_sent_with_num
0,about the authorjp buntinx is a fintech and bi...,0001-11-30 00:00:00+00:00,[],"[authorjp, buntinx, fintech, bitcoin, enthusia...","(0, 127102)\t0.22856558085345463\n (0, 1268...","[authorjp, buntinx, fintech, bitcoin, enthusia...","[authorjp, buntinx, fintech, bitcoin, enthusia...","[[(authorjp, NN)], [(buntinx, NN)], [(fintech,...",[],[]
1,new data shows chinese exchanges have accounte...,0001-11-30 00:00:00+00:00,[new data shows chinese exchanges have account...,"[new, data, shows, chinese, exchanges, account...","(0, 128858)\t0.05043681888099185\n (0, 1283...","[new, data, show, chines, exchang, account, bi...","[new, data, show, chinese, exchange, accounted...","[[(new, JJ)], [(data, NNS)], [(shows, NNS)], [...","[[new, data, shows, chinese, exchanges, have, ...","[[(new, JJ), (data, NN), (shows, NNS), (chines..."
2,a major european banking trade group has calle...,0001-11-30 00:00:00+00:00,[],"[major, european, banking, trade, group, calle...","(0, 128167)\t0.04010913921090279\n (0, 1257...","[major, european, bank, trade, group, call, go...","[major, european, banking, trade, group, calle...","[[(major, JJ)], [(european, JJ)], [(banking, N...",[],[]


In [5]:
#Create Sentiment Dataframe and Add Dates and Sentences with Numbers Column from bitcoin dataframe
sentiment_df_with_num = pd.DataFrame(columns=['date','article','article_polarity_neg_pos','article_subjectivity'])

sentiment_df_without_num = pd.DataFrame(columns=['date','article','article_polarity_neg_pos','article_subjectivity'])

#Assign Date and Sent With Number Values
sentiment_df_with_num.date = bitcoin_news_df['date']
sentiment_df_without_num .date = bitcoin_news_df['date']

sentiment_df_with_num.article = bitcoin_news_df['sent_with_num']
sentiment_df_without_num.article = bitcoin_news_df['text']

#Join sentences with number into single paragraph
sentiment_df_with_num.article = sentiment_df_with_num.article.apply(lambda x: ' '.join(x))

In [21]:
#Remove All Rows That Don't Have Text and Reset Index
sentiment_df_with_num = sentiment_df_with_num[sentiment_df_with_num['article'] != '']
sentiment_df_with_num = sentiment_df_with_num.reset_index(drop=True)


sentiment_df_without_num = sentiment_df_without_num[sentiment_df_without_num['article'] != '']
sentiment_df_without_num = sentiment_df_without_num.reset_index(drop=True)

In [25]:
def sentiment_parser(corpus):
    total_sentiment = []
    for line in corpus:
        line_sent=TextBlob(line).sentiment.polarity
        total_sentiment.append(line_sent)
    return total_sentiment

In [34]:
def subjectivity_parser(corpus):
    total_sentiment = []
    for line in corpus:
        line_sent=TextBlob(line).sentiment.subjectivity
        total_sentiment.append(line_sent)
    return total_sentiment

In [35]:
#Calculate Sentiment of Articles with Numbers
sentiment_articles_with_num = sentiment_parser(sentiment_df_with_num.article)

#Calculate Polarity of Articles with Numbers
subjectivity_articles_with_num = subjectivity_parser(sentiment_df_with_num.article)

In [ ]:
#Calculate Sentiment of Articles with Numbers
sentiment_articles_without_num = sentiment_parser(sentiment_df_without_num.article)

#Calculate Polarity of Articles with Numbers
subjectivity_articles_without_num = subjectivity_parser(sentiment_df_without_num.article)